In [3]:
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
#from pmdarima import auto_arima
from sklearn import metrics
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
warnings.filterwarnings("ignore")



In [4]:
#Let’s take a sneak peek at the data.
df =  pd.read_pickle('C:\\Users\\z0047cfr\\Project Work\\Factory Demand Model\\Input Data\\df_t.pkl')
df.index.freq = 'MS'
df['DATE']=df.index
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.loc[df.DATE >= '2015-12-01']
df.head(5)

,data,US quarterly transport equipment sales from Census,FU_Avg_Plant_hpw_all_mfg_Q_Census,FU_Avg_Plant_hpw_metal_mfg_Q_Census,FU_Avg_Plant_hpw_fab_metal_mfg_Q_Census,FU_Avg_Plant_hpw_Metalwrk_mach_mfg_Q_Census,FU_Avg_Plant_hpw_railroad_roll_stk_mfg_Q_Census,EU_Avg_Plant_hpw_all_mfg_Q_Census,EU_Avg_Plant_hpw_metal_mfg_Q_Census,EU_Avg_Plant_hpw_fab_metal_mfg_Q_Census,...,wage_min_scl,wage_max_scl,Orders_millions_Q_scl,Revenue_millions_Q_scl,Profit_Q_scl,Severance_Amount_millions_Q_scl,Profit_margin_perc_Q_scl,Profit_post_severance_perc_Q_scl,Labor_conv_count_M_scl,DATE
2015-12-01,56.0,293912.0,70.6,63.3,62.6,72.2,54.0,54.6,56.9,39.0,...,-1.961777,-0.332421,0.203158,-0.441984,-0.277008,-0.516869,-0.133665,-0.196923,0.298102,2015-12-01
2016-01-01,56.0,281628.0,71.7,69.6,64.5,65.3,58.1,56.2,63.5,41.3,...,0.327654,-1.895609,-0.904430,-1.207205,-1.346161,-0.516869,-1.240350,-1.320113,-0.521679,2016-01-01
2016-02-01,59.0,281628.0,71.7,69.6,64.5,65.3,58.1,56.2,63.5,41.3,...,0.327654,-1.895609,-0.904430,-1.207205,-1.346161,-0.516869,-1.240350,-1.320113,-0.521679,2016-02-01
2016-03-01,60.0,281628.0,71.7,69.6,64.5,65.3,58.1,56.2,63.5,41.3,...,-1.961777,-0.332421,-0.904430,-1.207205,-1.346161,-0.516869,-1.240350,-1.320113,-0.521679,2016-03-01
2016-04-01,64.0,297398.0,72.6,73.6,62.4,69.6,60.1,56.8,66.9,41.0,...,0.327654,-1.106276,-1.846805,-1.919038,-1.212517,-0.347830,-0.607958,-0.598062,-0.521679,2016-04-01


In [5]:
df = df.replace(np.inf, np.nan).replace(-np.inf, np.nan)
df = df.interpolate(method='linear')
df = df.interpolate(method='ffill')
df = df.interpolate(method='bfill')
df.isna().sum().sum()
#df2.data.head(), df2.data.tail()

0

In [6]:
#Define the time-series evaluation function, as shown here:

def timeseries_evaluation_metrics_func(y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true,y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true,y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true,y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def get_rmse(y, y_hat):
    mse = np.mean((y - y_hat)**2)
    return np.sqrt(mse)

def get_mape(y, y_hat):
    perc_err = (100*(y - y_hat))/y
    return np.mean(abs(perc_err))

def get_mase(y, y_hat):

    abs_err = abs(y - y_hat)
    dsum=sum(abs(y[1:] - y_hat[1:]))
    t = len(y)
    denom = (1/(t - 1))* dsum
    return np.mean(abs_err/denom)

In [7]:
#ADF test function to check for stationary data:

def Augmented_Dickey_Fuller_Test_func(series , column_name):
    print (f'Results of Dickey-Fuller Test for column:{column_name}')
    dftest = adfuller(series, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','No Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    if dftest[1] <= 0.05:
        print("Conclusion:====>")
        print("Reject the null hypothesis")
        print("Data is stationary")
    else:
        print("Conclusion:====>")
        print("Fail to reject the null hypothesis")
        print("Data is non-stationary")

In [8]:
train_start =  0
train_end = len(df) - 12
train = df.iloc[train_start: train_end]

test_start = len(train)
test_end = len(train) + 9
test = df.iloc[len(train):len(train)+ 9] # 9 months of test post training


train_test = pd.concat([train, test], axis=0) # develop the model to retrain using train and test as development sample

val_start = len(train) + len(test) 
val_end =  len(train) +   len(test) + 3
val = df.iloc[len(train) + len(test): len(train) + len(test) + 3] # 3 months of validation post training+test 

#train_start , train_end , test_start, test_end, val_start, val_end 
len(df), len(train), len(test), len(val), len(train_test)


(55, 43, 9, 3, 52)

In [20]:
df.iloc[:4, 0:8].head()
#	DF1.iloc[: ,4]


,data,US quarterly transport equipment sales from Census,FU_Avg_Plant_hpw_all_mfg_Q_Census,FU_Avg_Plant_hpw_metal_mfg_Q_Census,FU_Avg_Plant_hpw_fab_metal_mfg_Q_Census,FU_Avg_Plant_hpw_Metalwrk_mach_mfg_Q_Census,FU_Avg_Plant_hpw_railroad_roll_stk_mfg_Q_Census,EU_Avg_Plant_hpw_all_mfg_Q_Census
2015-12-01,56.0,293912.0,70.6,63.3,62.6,72.2,54.0,54.6
2016-01-01,56.0,281628.0,71.7,69.6,64.5,65.3,58.1,56.2
2016-02-01,59.0,281628.0,71.7,69.6,64.5,65.3,58.1,56.2
2016-03-01,60.0,281628.0,71.7,69.6,64.5,65.3,58.1,56.2


In [30]:
# var_list = [
#  'Severance_Amount_millions_Q_s3',
#  'Labor_conv_count_M_lag',
#  'EU_Avg_Plant_hpw_all_mfg_Q_Census_d3',
#  'FU_Avg_Plant_hpw_all_mfg_Q_Census_log',
#  'Severance_Amount_millions_Q_d3',
# 'Profit_post_severance_perc_Q_d1',
#  'Unemp_M_CA_Fred_d3'
# ]
var_list = [
'Profit_post_severance_perc_Q_d1'
]
    

# df[var_list].describe()

In [31]:
exog_tr = train[var_list]
exog_tr2 = train_test[var_list]
exog_te = test[var_list]
exog_val = val[var_list]
exog_te.isna().sum().sum(), exog_tr.isna().sum().sum()
#exog_tr.shape
#exog_te.shape
#test.shape
#train.shape

(0, 0)

# Grid Search

In [32]:
# Grid search SARIMAX
#  Grid Search
import itertools
#set parameter range
p = range(0,3)
q = range(0,3)
d = [1]
s = [9]
Final_1 = pd.DataFrame(columns= ['Order', 'Seasonal_Order', 'MSE','MAPE', 'AIC'])
dat = pd.DataFrame()
# list of all parameter combos
pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(p, d, q, s))
# SARIMA model pipeline
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            order=[]
            seasonal_order=[]
            #Errors=[]
            AIC=[]
            mod = sm.tsa.statespace.SARIMAX(train.data, exog = exog_tr,
                                    order=param,seasonal_order=param_seasonal)
            results = mod.fit()
            
            predictions = results.predict(start=test_start, end=test_end-1, exog = exog_te, dynamic=False, typ='levels').rename('forecast')
            test_pred = predictions.to_frame()
            test1 = pd.merge(test, test_pred, how = 'outer',left_index=True, right_index=True) 
                      
            
            from sklearn.metrics import mean_squared_error
            #print(test1[['data','forecast']].head(2))
                       
            MSE = metrics.mean_absolute_error(test1.data, test1.forecast)
            def mean_absolute_percentage_error(y_true, y_pred):
               y_true, y_pred = np.array(y_true), np.array(y_pred)
               return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
            MAPE = mean_absolute_percentage_error(test1.data, test1.forecast)
            
            print('SARIMAX{},{} - AIC:{}, Test MAPE:{}'.format(param, param_seasonal, results.aic.round(), MAPE.round()))
            cols = ['Order', 'Seasonal_Order', 'MSE','MAPE', 'AIC']
            dat = pd.DataFrame(columns = cols)
            dat = dat.append({'Order': param, 'Seasonal_Order':param_seasonal,'MSE': MSE,'MAPE':MAPE,'AIC':results.aic},ignore_index=True)

            Final_1= Final_1.append(dat)
            #print("Dataframe : " , dat, sep='\n')
        except:
            print("Code didnt run")
            continue


SARIMAX(0, 1, 0),(0, 1, 0, 9) - AIC:263.0, Test MAPE:30.0
SARIMAX(0, 1, 0),(0, 1, 1, 9) - AIC:260.0, Test MAPE:10.0
SARIMAX(0, 1, 0),(0, 1, 2, 9) - AIC:257.0, Test MAPE:7.0
SARIMAX(0, 1, 0),(1, 1, 0, 9) - AIC:260.0, Test MAPE:9.0
SARIMAX(0, 1, 0),(1, 1, 1, 9) - AIC:260.0, Test MAPE:5.0
SARIMAX(0, 1, 0),(1, 1, 2, 9) - AIC:259.0, Test MAPE:8.0
SARIMAX(0, 1, 0),(2, 1, 0, 9) - AIC:258.0, Test MAPE:9.0
SARIMAX(0, 1, 0),(2, 1, 1, 9) - AIC:257.0, Test MAPE:11.0
SARIMAX(0, 1, 0),(2, 1, 2, 9) - AIC:259.0, Test MAPE:9.0
SARIMAX(0, 1, 1),(0, 1, 0, 9) - AIC:256.0, Test MAPE:32.0
SARIMAX(0, 1, 1),(0, 1, 1, 9) - AIC:254.0, Test MAPE:14.0
SARIMAX(0, 1, 1),(0, 1, 2, 9) - AIC:252.0, Test MAPE:4.0
SARIMAX(0, 1, 1),(1, 1, 0, 9) - AIC:254.0, Test MAPE:12.0
SARIMAX(0, 1, 1),(1, 1, 1, 9) - AIC:255.0, Test MAPE:7.0
SARIMAX(0, 1, 1),(1, 1, 2, 9) - AIC:254.0, Test MAPE:5.0
SARIMAX(0, 1, 1),(2, 1, 0, 9) - AIC:253.0, Test MAPE:6.0
SARIMAX(0, 1, 1),(2, 1, 1, 9) - AIC:253.0, Test MAPE:10.0
SARIMAX(0, 1, 1),(2, 1, 

In [41]:
Final_1.to_csv('SARIMAX_Gridsearch_NB_Profit_post_severance_perc_Q_d1.csv', index = False, header=True)

In [ ]:
# SARIMAX(0, 1, 1),(0, 1, 2, 9) - AIC:252.0, Test MAPE:4.0 for var 'Profit_post_severance_perc_Q_d1'

# Final Chosen Parameters

In [43]:
param = (0, 1, 1)
param_seasonal =  (0, 1, 2, 9)
#(0, 1, 1)	(0, 1, 2, 9)


# Running the best SARIMA Model post grid search on train

In [44]:


# fit ARIMA monthly based on helper plots
model2 = sm.tsa.statespace.SARIMAX(train.data, exog = exog_tr,
                                    order=param,seasonal_order=param_seasonal).fit()
print(model2.summary())
# Obtain predicted values

predictions = model2.predict(start=test_start, end=test_end-1, exog = exog_te, dynamic=False, typ='levels').rename('forecast')
test_pred = predictions.to_frame()
test1 = pd.merge(test, test_pred, how = 'outer',left_index=True, right_index=True) 
print("MAPE: ",get_mape(test1.data, test1.forecast))    
print("MSE: ", mean_squared_error(test1.data, test1.forecast))
print("MAE: ", mean_absolute_error(test1.data, test1.forecast))

                                       SARIMAX Results                                        
Dep. Variable:                                   data   No. Observations:                   43
Model:             SARIMAX(0, 1, 1)x(0, 1, [1, 2], 9)   Log Likelihood                -120.945
Date:                                Fri, 04 Sep 2020   AIC                            251.890
Time:                                        17:51:26   BIC                            259.373
Sample:                                    12-01-2015   HQIC                           254.408
                                         - 06-01-2019                                         
Covariance Type:                                  opg                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Profit_post_severance_perc_Q_d1    -0.33

# Retraining and FItting the Final Model on Validation set

In [45]:
# fit ARIMA monthly based on helper plots
model1_val = sm.tsa.statespace.SARIMAX(train_test.data, exog = exog_tr2,
                                    order=param,seasonal_order=param_seasonal).fit()
print(model1_val.summary())
# Obtain predicted values

predictions_val = model1_val.predict(start=val_start, end=val_end-1, exog = exog_val, dynamic=False, typ='levels').rename('forecast')
val_pred = predictions_val.to_frame()
val1 = pd.merge(val, val_pred, how = 'outer',left_index=True, right_index=True) 
print("MAPE: ", get_mape(val1.data, val1.forecast))    
print("MSE: ",  mean_squared_error(val1.data, val1.forecast))
print("MAE: ",  mean_absolute_error(val1.data, val1.forecast))

                                       SARIMAX Results                                        
Dep. Variable:                                   data   No. Observations:                   52
Model:             SARIMAX(0, 1, 1)x(0, 1, [1, 2], 9)   Log Likelihood                -152.197
Date:                                Fri, 04 Sep 2020   AIC                            314.394
Time:                                        17:51:30   BIC                            323.082
Sample:                                    12-01-2015   HQIC                           317.578
                                         - 03-01-2020                                         
Covariance Type:                                  opg                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Profit_post_severance_perc_Q_d1    -0.37

# getting fit on training data - keeps a check if the Model is overfitting

In [46]:
train_start

0

In [47]:
# Obtain predicted values on train data
start=1
end=len(train)+len(test)-1
predictions_train = model2.predict(start=train_start, end=train_end-1, dynamic=False, typ='levels', exog = exog_tr).rename('forecast')
predictions_train
train_pred = predictions_train.to_frame()
train1 = pd.merge(train, train_pred, how = 'left',left_index=True, right_index=True)
#train1 = train1.iloc[1:]
train1[['data','forecast']].head()

,data,forecast
2015-12-01,56.0,-5.422001
2016-01-01,56.0,61.898345
2016-02-01,59.0,55.525206
2016-03-01,60.0,59.000217
2016-04-01,64.0,59.695075


In [48]:
# test metrics

print("Test Model AIC : ", model2.aic, "       Validation Model AIC : ", model1_val.aic)
print("Test Model  : ", model2.bic, "          Validation Model  : ", model1_val.bic)

print("Test MSE: ", mean_squared_error(test1.data, test1.forecast),        "     Val MSE: ", mean_squared_error(val1.data, val1.forecast), "    Train MSE: ", mean_squared_error(train1.data, train1.forecast))
print("Test MAE: ", mean_absolute_error(test1.data, test1.forecast),       "     Val MAE: ", mean_absolute_error(val1.data, val1.forecast), "    Train MAE: ", mean_absolute_error(train1.data, train1.forecast))
print("Test RMSE: ", get_rmse(test1.data, test1.forecast),                  "    Val RMSE: ", get_rmse(val1.data, val1.forecast), "           Train RMSE: ", get_rmse(train1.data, train1.forecast))
print("Test MAPE: ", get_mape(test1.data, test1.forecast),                  "    Val MAPE: ", get_mape(val1.data, val1.forecast), "           Train MAPE: ", get_mape(train1.data, train1.forecast))
print("Test MASE: ", get_mase(test1.data, test1.forecast),                  "    Val MASE: ", get_mase(val1.data, val1.forecast), "                     Train MASE: ", get_mase(train1.data, train1.forecast))

Test Model AIC :  251.89012952007104        Validation Model AIC :  314.393764592871
Test Model  :  259.37266732740346           Validation Model  :  323.08211268428784
Test MSE:  28.13851551379421      Val MSE:  389.1886161957785     Train MSE:  192.0077558557449
Test MAE:  4.522169275378076      Val MAE:  19.174310951614117     Train MAE:  9.353134985153815
Test RMSE:  5.304574960710256     Val RMSE:  19.727863954208992            Train RMSE:  13.856686323062409
Test MAPE:  4.129583234094695     Val MAPE:  20.48405668117429            Train MAPE:  11.713542180177978
Test MASE:  1.029059664157006     Val MASE:  0.9422892918076547                      Train MASE:  1.1528009099751484
